# Strategy analysis example

Debugging a strategy can be time-consuming. Freqtrade offers helper functions to visualize raw data.
The following assumes you work with SampleStrategy, data for 5m timeframe from Binance and have downloaded them into the data directory in the default location.
Please follow the [documentation](https://www.freqtrade.io/en/stable/data-download/) for more details.

### Configure Freqtrade environment

In [13]:
from freqtrade.configuration import Configuration
from freqtrade.data.history import load_pair_history
from freqtrade.enums import CandleType
from pathlib import Path
import talib.abstract as ta

cwd = Path().cwd().parent

config = Configuration.from_files([cwd / "config.json"])

config["strategy"] = "TrendMomoClassifier"
config["freqaimodel"] = "CatboostClassifier"
config["timerange"] = "20210201-20230212"

pair="ETH/USDT:USDT"

datadir = cwd / rf"data/{ config['exchange']['name'] }"

candles = load_pair_history(pair,
                            config["timeframe"],
                            datadir,
                            data_format = "hdf5",  # Make sure to update this to your data
                            candle_type=CandleType.FUTURES)

candles.tail()

,date,open,high,low,close,volume
74633,2023-02-17 10:15:00+00:00,1664.15,1666.50,1663.25,1663.75,214733.0
74634,2023-02-17 10:30:00+00:00,1663.75,1666.80,1662.20,1666.55,193660.0
74635,2023-02-17 10:45:00+00:00,1666.60,1667.00,1663.20,1664.30,160458.0
74636,2023-02-17 11:00:00+00:00,1664.30,1665.65,1663.60,1664.60,121594.0
74637,2023-02-17 11:15:00+00:00,1664.65,1669.40,1664.50,1668.85,339947.0


## Load and run strategy
* Rerun each time the strategy file is changed

In [14]:
from freqtrade.resolvers import StrategyResolver
from freqtrade.freqai.data_kitchen import FreqaiDataKitchen
from freqtrade.data.dataprovider import DataProvider

strategy = StrategyResolver.load_strategy(config)

strategy.load_freqAI_model()

strategy.dp = DataProvider(config, None, None)

df = strategy.analyze_ticker(candles, {'pair': pair})

df.tail()

,date,open,high,low,close,volume,&s-up_or_down,down,down_mean,down_std,...,up_std,do_predict,DI_values,stoch_rsi_K,stoch_rsi_D,stoploss_rate,enter_long,enter_short,exit_long,exit_short
74633,2023-02-17 10:15:00+00:00,1664.15,1666.50,1663.25,1663.75,214733.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,68.831381,70.470892,1652.080345,NaN,NaN,1.0,NaN
74634,2023-02-17 10:30:00+00:00,1663.75,1666.80,1662.20,1666.55,193660.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,87.512165,1655.056749,NaN,NaN,NaN,NaN
74635,2023-02-17 10:45:00+00:00,1666.60,1667.00,1663.20,1664.30,160458.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.661804,58.497729,1653.084839,NaN,NaN,1.0,NaN
74636,2023-02-17 11:00:00+00:00,1664.30,1665.65,1663.60,1664.60,121594.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,11.258925,39.306910,1653.893064,NaN,NaN,NaN,NaN
74637,2023-02-17 11:15:00+00:00,1664.65,1669.40,1664.50,1668.85,339947.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,39.306910,1658.207846,NaN,NaN,NaN,NaN


### Display the trade details

In [15]:
# Report results
print(f"Generated {df['enter_long'].sum()} entry signals")
data = df.set_index('date', drop=False)
data.tail()

Generated 482.0 entry signals


,date,open,high,low,close,volume,&s-up_or_down,down,down_mean,down_std,...,up_std,do_predict,DI_values,stoch_rsi_K,stoch_rsi_D,stoploss_rate,enter_long,enter_short,exit_long,exit_short
date,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,2021-01-01 00:00:00+00:00,736.65,739.80,730.70,731.55,104408.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:15:00+00:00,2021-01-01 00:15:00+00:00,731.70,732.90,730.70,732.30,65479.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:30:00+00:00,2021-01-01 00:30:00+00:00,732.40,734.95,732.40,734.25,24503.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 00:45:00+00:00,2021-01-01 00:45:00+00:00,734.25,736.35,733.45,734.90,40830.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-01 01:00:00+00:00,2021-01-01 01:00:00+00:00,735.15,744.10,734.60,744.10,82430.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-17 10:15:00+00:00,2023-02-17 10:15:00+00:00,1664.15,1666.50,1663.25,1663.75,214733.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,68.831381,70.470892,1652.080345,NaN,NaN,1.0,NaN
2023-02-17 10:30:00+00:00,2023-02-17 10:30:00+00:00,1663.75,1666.80,1662.20,1666.55,193660.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,87.512165,1655.056749,NaN,NaN,NaN,NaN
2023-02-17 10:45:00+00:00,2023-02-17 10:45:00+00:00,1666.60,1667.00,1663.20,1664.30,160458.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.661804,58.497729,1653.084839,NaN,NaN,1.0,NaN


## Load Existing Backtest Data 

In [16]:
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats

backtest_dir = config["user_data_dir"] / "backtest_results"
# backtest_dir = backtest_dir / "backtest-result-2020-07-01_20-04-22.json"

# You can get the full backtest statistics by using the following command.
# This contains all information used to generate the backtest result.
stats = load_backtest_stats(backtest_dir)

# All statistics are available per strategy, so if `--strategy-list` was used during backtest, this will be reflected here as well.
# Example usages:
print(stats['strategy'][config["strategy"]]['results_per_pair'])
# Get pairlist used for this backtest
print(stats['strategy'][config["strategy"]]['pairlist'])
# Get market change (average change of all pairs from start to end of the backtest period)
print(stats['strategy'][config["strategy"]]['market_change'])
# Maximum drawdown ()
print(stats['strategy'][config["strategy"]]['max_drawdown'])
# Maximum drawdown start and end
print(stats['strategy'][config["strategy"]]['drawdown_start'])
print(stats['strategy'][config["strategy"]]['drawdown_end'])


# Get strategy comparison (only relevant if multiple strategies were compared)
print(stats['strategy_comparison'])


[{'key': 'ETH/USDT:USDT', 'trades': 996, 'profit_mean': -0.000828163801179021, 'profit_mean_pct': -0.0828163801179021, 'profit_sum': -0.824851145974305, 'profit_sum_pct': -82.49, 'profit_total_abs': -153.11495696, 'profit_total': -0.15311495696, 'profit_total_pct': -15.31, 'duration_avg': '0:51:00', 'wins': 408, 'draws': 0, 'losses': 588}, {'key': 'TOTAL', 'trades': 996, 'profit_mean': -0.000828163801179021, 'profit_mean_pct': -0.0828163801179021, 'profit_sum': -0.824851145974305, 'profit_sum_pct': -82.49, 'profit_total_abs': -153.11495696, 'profit_total': -0.15311495696, 'profit_total_pct': -15.31, 'duration_avg': '0:51:00', 'wins': 408, 'draws': 0, 'losses': 588}]
['ETH/USDT:USDT']
0.24854955680902505
0.8716554492130072
2021-02-09 20:45:00
2023-02-14 04:15:00
[{'key': 'TrendMomoClassifier', 'trades': 996, 'profit_mean': -0.000828163801179021, 'profit_mean_pct': -0.0828163801179021, 'profit_sum': -0.824851145974305, 'profit_sum_pct': -82.49, 'profit_total_abs': -153.11495696, 'profit_

In [17]:
# Load backtested trades as dataframe
trades = load_backtest_data(backtest_dir)

# Show value-counts per pair
trades.groupby("pair")["exit_reason"].value_counts()

pair           exit_reason       
ETH/USDT:USDT  trailing_stop_loss    585
               exit_signal           405
               stop_loss               6
Name: exit_reason, dtype: int64

## Plotting daily profit / equity line

In [18]:
# Plotting equity line (starting with 0 on day 1 and adding daily profit for each backtested day)

from freqtrade.configuration import Configuration
from freqtrade.data.btanalysis import load_backtest_data, load_backtest_stats
import plotly.express as px
import pandas as pd

# strategy = 'SampleStrategy'
# config = Configuration.from_files(["user_data/config.json"])
# backtest_dir = config["user_data_dir"] / "backtest_results"

stats = load_backtest_stats(backtest_dir)
strategy_stats = stats['strategy'][config["strategy"]]

dates = []
profits = []
for date_profit in strategy_stats['daily_profit']:
    dates.append(date_profit[0])
    profits.append(date_profit[1])

equity = 0
equity_daily = []
for daily_profit in profits:
    equity_daily.append(equity)
    equity += float(daily_profit)


df = pd.DataFrame({'dates': dates,'equity_daily': equity_daily})

fig = px.line(df, x="dates", y="equity_daily")
fig.show()


### Load live trading results

In [19]:
from freqtrade.data.btanalysis import load_trades_from_db

# Fetch trades from database
trades = load_trades_from_db("sqlite:///tradesv3.sqlite")

# Display results
trades.groupby("pair")["exit_reason"].value_counts()

Series([], Name: exit_reason, dtype: int64)

## Analyze the loaded trades for trade parallelism
This can be useful to find the best `max_open_trades` parameter, when used with backtesting in conjunction with `--disable-max-market-positions`.

In [20]:
from freqtrade.data.btanalysis import analyze_trade_parallelism

# Analyze the above
parallel_trades = analyze_trade_parallelism(trades, '5m')

parallel_trades.plot()

ValueError: No objects to concatenate

## Plot results

Freqtrade offers interactive plotting capabilities based on plotly.

In [ ]:
from freqtrade.plot.plotting import  generate_candlestick_graph
# Limit graph period to keep plotly quick and reactive

# Filter trades to one pair
trades_red = trades.loc[trades['pair'] == pair]

data_red = data['2019-06-01':'2019-06-10']
# Generate candlestick graph
graph = generate_candlestick_graph(pair=pair,
                                   data=data_red,
                                   trades=trades_red,
                                   indicators1=['sma20', 'ema50', 'ema55'],
                                   indicators2=['rsi', 'macd', 'macdsignal', 'macdhist']
                                  )




In [ ]:
# Show graph inline
# graph.show()

# Render graph in a separate window
graph.show(renderer="browser")


## Plot average profit per trade as distribution graph

In [ ]:
import plotly.figure_factory as ff

hist_data = [trades.profit_ratio]
group_labels = ['profit_ratio']  # name of the dataset

fig = ff.create_distplot(hist_data, group_labels, bin_size=0.01)
fig.show()


ValueError: min() arg is an empty sequence